Use this utlity to update the returns and std_dev fields within investment-options.csv

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [3]:
# Set size of inline plots.
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [5]:
# Read in investment options input csv.
etfs = pd.read_csv('investment-options-in.csv', skip_blank_lines=True, comment='#')
etfs = list(etfs['Investment Option'])

In [6]:
# Eliminate repeat symbols.
symbols = set(etfs)

In [7]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

..........................................................................


In [8]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)

In [ ]:
# Sample symbol.
symbol = 'SPY'

In [9]:
annual_returns = bb.annualized_returns(df, timeperiod='daily', years=1)
# Sample
annual_returns[symbol]

6.30089917255674

In [10]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(df, timeperiod='daily', years=5)

In [11]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola[symbol]

0.16334637699461557

In [12]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola['SPY']

0.09612992755410756

In [13]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('M').ffill()
bb.print_full(monthly[symbol])

Date
2015-01-31   174.55
2015-02-28   184.36
2015-03-31   181.47
2015-04-30   183.25
2015-05-31   185.61
2015-06-30   181.84
2015-07-31   185.94
2015-08-31   174.61
2015-09-30   170.15
2015-10-31   184.63
2015-11-30   185.30
2015-12-31   182.10
2016-01-31   173.03
2016-02-29   172.89
2016-03-31   184.52
2016-04-30   185.25
2016-05-31   188.40
2016-06-30   189.05
2016-07-31   195.95
2016-08-31   196.18
2016-09-30   196.20
2016-10-31   192.79
2016-11-30   199.90
2016-12-31   203.95
2017-01-31   207.60
2017-02-28   215.76
2017-03-31   216.03
2017-04-30   218.17
2017-05-31   221.25
2017-06-30   222.66
2017-07-31   227.24
2017-08-31   227.90
2017-09-30   232.49
2017-10-31   237.97
2017-11-30   245.24
2017-12-31   248.22
2018-01-31   262.21
2018-02-28   252.67
2018-03-31   245.75
2018-04-30   247.02
2018-05-31   253.02
2018-06-30   254.48
2018-07-31   263.90
2018-08-31   272.33
2018-09-30   273.95
2018-10-31   255.02
2018-11-30   259.75
2018-12-31   236.88
2019-01-31   255.84
2019-02-28   26

In [14]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()
monthly_returns[symbol]

Date
2015-01-31     NaN
2015-02-28    0.06
2015-03-31   -0.02
2015-04-30    0.01
2015-05-31    0.01
              ... 
2021-12-31    0.05
2022-01-31   -0.05
2022-02-28   -0.03
2022-03-31    0.04
2022-04-30   -0.03
Freq: M, Name: SPY, Length: 88, dtype: float64

In [15]:
# Calculate 3 year annualized standard deviation.
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev[symbol]

0.17587667626569933

In [16]:
# Read investment-options-in.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
#lines

In [17]:
# For each etf, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = []
for line in lines:
    # Copy empty and comment lines.
    if not line or line.startswith('#'):
        out.append(line)
        continue
    # Split line by comma delimiter; strip the fields.
    fields = line.split(',')
    fields = [field.strip() for field in fields]
    symbol = fields[0].strip('\"')
    # Copy header.
    if symbol == 'Investment Option':
        out.append(line)
        continue
    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append('{},{},{},"{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
               .format(fields[0], fields[1], fields[2],
                       ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [18]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')